# Points of Interest

## Imports and Setup

In [9]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

import logging
from pathlib import Path

from icecream import ic

from IPython.display import display
import ipywidgets as wid
from utils.ipywidgets_extended import widgets_styling

from utils.setup_notebook import init_notebook
from utils.setup_logging import setup_logging
import utils.memoize as memoize

init_notebook()
setup_logging("INFO")
memoize.set_file_store_path("points_of_interest")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import numpy as np
import pandas as pd
import scipy as sp
import numba as nb
from numba import cuda
import cv2

from utils.benchmarking import LogTimer
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_kernel,
    plot_matrix,
)
from utils.image_tools import load_image, LoadedImage
import utils.dyn_module as dyn
from utils.cv2_tools import draw_keypoints, draw_matches

logging.getLogger("numba.cuda.cudadrv.driver").setLevel(logging.WARNING)

In [11]:
reset_memoize_store_button = wid.Button(description="Reset memoize store")
reset_memoize_store_button.on_click(lambda x: memoize.reset_store())
display(reset_memoize_store_button)


Button(description='Reset memoize store', style=ButtonStyle())

## Loading Points of Interest Implementations

In [12]:
dir_points_of_interest_impls = "./points_of_interest_impls"
points_of_interest_impls_module_names = dyn.load_modules(dir_points_of_interest_impls)


2024-11-06 13:53:42.953 INFO root Loading 1 modules started (..\utils\dyn_module.py:59)
2024-11-06 13:53:42.975 INFO root Reloading harris_points_of_interest started (..\utils\dyn_module.py:26)
2024-11-06 13:53:43.017 INFO root Reloading harris_points_of_interest took: 31.8552 ms (..\utils\dyn_module.py:26)
2024-11-06 13:53:43.050 INFO root Loading 1 modules took: 108.0505 ms (..\utils\dyn_module.py:59)


## Loading Input Image Sets

In [13]:
input_image_set_dir = "./image_sets_input"
input_image_set_scaled_dir = "./image_sets_scaled_input"

input_image_sets = {}


def load_image_sets(image_sets_dir: str):
    image_sets_folders = os.listdir(image_sets_dir)
    image_sets_folders.sort()

    with LogTimer(f"Loading image sets from {Path(image_sets_dir).name}"):
        for image_set_name in image_sets_folders:
            with LogTimer(f"Loading image set {image_set_name}"):
                image_set_dir = os.path.join(image_sets_dir, image_set_name)
                image_set = []

                images_in_image_set = os.listdir(image_set_dir)
                images_in_image_set.sort()

                for image_name in images_in_image_set:
                    image = load_image(os.path.join(image_set_dir, image_name))
                    image_set.append(image)

                input_image_sets[image_set_name] = image_set


def save_image_set(image_set: list, image_set_dir: str):
    os.makedirs(image_set_dir, exist_ok=True)
    for image in image_set:
        cv2.imwrite(os.path.join(image_set_dir, image.filename), image.image_color)


load_image_sets(input_image_set_dir)
load_image_sets(input_image_set_scaled_dir)

2024-11-06 13:53:43.347 INFO root Loading image sets from image_sets_input started (notebook_cell:11)
2024-11-06 13:53:43.402 INFO root Loading image set the_office started (notebook_cell:13)
2024-11-06 13:53:43.458 INFO root Loading 20241105_124827.jpg started (..\utils\image_tools.py:69)
2024-11-06 13:53:44.263 INFO root Loading 20241105_124827.jpg took: 816.4598 ms (..\utils\image_tools.py:69)
2024-11-06 13:53:44.290 INFO root Loading 20241105_124828.jpg started (..\utils\image_tools.py:69)
2024-11-06 13:53:45.014 INFO root Loading 20241105_124828.jpg took: 724.8113 ms (..\utils\image_tools.py:69)
2024-11-06 13:53:45.033 INFO root Loading 20241105_124831.jpg started (..\utils\image_tools.py:69)
2024-11-06 13:53:45.589 INFO root Loading 20241105_124831.jpg took: 548.4310 ms (..\utils\image_tools.py:69)
2024-11-06 13:53:45.623 INFO root Loading 20241105_124833.jpg started (..\utils\image_tools.py:69)
2024-11-06 13:53:46.261 INFO root Loading 20241105_124833.jpg took: 628.2812 ms (..\u

## Image set scaler

In [14]:
def get_largest_image_in_image_set(image_set: list) -> np.array:
    largest_image = None
    largest_pixel_count = 0
    for image in image_set:
        resolution = image.image_color.shape[:2]
        pixel_count = resolution[0] * resolution[1]
        if pixel_count > largest_pixel_count:
            largest_pixel_count = pixel_count
            largest_image = image
    return largest_image


def get_largest_resolution_in_image_set(image_set: list) -> tuple:
    largest_image = get_largest_image_in_image_set(image_set)
    return largest_image.image_color.shape[:2]

In [41]:
KEY_SCALER_IMAGE_SET_DROPDOWN = "scaler_image_set_dropdown"
scaler_image_set_dropdown = wid.Dropdown(
    options=list(input_image_sets.keys()),
    value=memoize.get(
        KEY_SCALER_IMAGE_SET_DROPDOWN,
        default=next(iter(input_image_sets.keys())),
        possible_values=input_image_sets.keys(),
    ),
    description="Image set",
    **widgets_styling,
)
scaler_largest_resolution_label = wid.Label("Largest image: (X,X)")
KEY_SCALER_SCALE_SLIDER = "scaler_scale_slider"
scaler_scale_slider = wid.FloatSlider(
    value=memoize.get(KEY_SCALER_SCALE_SLIDER, default=1.0),
    min=0.1,
    max=30.0,
    step=0.1,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
    description="Scale",
)
scaler_result_resolution_label = wid.Label("Resulting largest image: (X,X)")
scaler_create_scaled_image_set_button = wid.Button(
    description="Create scaled image set",
    **widgets_styling,
)


def on_update_resolution_labels(change=None):
    memoize.set(KEY_SCALER_IMAGE_SET_DROPDOWN, scaler_image_set_dropdown.value)
    memoize.set(KEY_SCALER_SCALE_SLIDER, scaler_scale_slider.value)

    image_set = input_image_sets[scaler_image_set_dropdown.value]
    largest_image = get_largest_image_in_image_set(image_set)
    largest_resolution = largest_image.image_color.shape[:2]
    scaler_largest_resolution_label.value = f"Largest image: {largest_resolution}"
    scaler_result_resolution_label.value = f"Resulting largest image: {tuple(int(x * 1/scaler_scale_slider.value) for x in largest_resolution)}"


scaler_scale_slider.observe(on_update_resolution_labels, names="value")
scaler_image_set_dropdown.observe(on_update_resolution_labels, names="value")
on_update_resolution_labels()


def create_scaled_image_set(change=None):
    with LogTimer(f"Creating scaled image set for {scaler_image_set_dropdown.value}"):
        original_image_set = input_image_sets[scaler_image_set_dropdown.value]

        scale = 1 / scaler_scale_slider.value

        largest_resolution = get_largest_resolution_in_image_set(original_image_set)
        scaled_resolution = tuple(int(x * scale) for x in largest_resolution)
        scaled_resolution_fs_string = f"({scaled_resolution[0]},{scaled_resolution[1]})"

        scaled_image_set_name = (
            f"{scaler_image_set_dropdown.value}_scaled_{scaled_resolution_fs_string}"
        )

        scaled_image_set_dir = os.path.join(
            input_image_set_scaled_dir, scaled_image_set_name
        )

        new_image_set = []
        for image in original_image_set:
            scaled_resolution = tuple(
                int(x * scale) for x in image.image_color.shape[:2]
            )
            scaled_resolution_fs_string = (
                f"({scaled_resolution[0]},{scaled_resolution[1]})"
            )
            with LogTimer(
                f"Resizing image {image.filename} from {image.image_color.shape[:2]} to {scaled_resolution}"
            ):
                new_image = LoadedImage()
                new_image.image_color = cv2.resize(
                    image.image_color, (scaled_resolution[1], scaled_resolution[0])
                )
                new_image.filename = f"{Path(image.filename).stem}_{scaled_resolution_fs_string}{Path(image.filename).suffix}"
                new_image_set.append(new_image)

        save_image_set(new_image_set, scaled_image_set_dir)
        load_image_sets(input_image_set_scaled_dir)


scaler_create_scaled_image_set_button.on_click(create_scaled_image_set)

display(
    wid.VBox(
        [
            wid.HBox([scaler_image_set_dropdown, scaler_largest_resolution_label]),
            wid.HBox([scaler_scale_slider, scaler_result_resolution_label]),
            scaler_create_scaled_image_set_button,
        ]
    )
)

## Running Points of Interest detection

In [56]:
fig_width = 16
image_size = 512

KEY_IMAGE_SET_DROPDOWN = "image_set_dropdown"
image_set_dropdown = wid.Dropdown(
    options=list(input_image_sets.keys()),
    value=memoize.get(
        KEY_IMAGE_SET_DROPDOWN,
        default=next(iter(input_image_sets.keys())),
        possible_values=input_image_sets.keys(),
    ),
    description="Image set",
    **widgets_styling,
)
KEY_POINTS_OF_INTEREST_IMPL_DROPDOWN = "points_of_interest_impl_dropdown"
points_of_interest_impl_dropdown = wid.Dropdown(
    options=points_of_interest_impls_module_names,
    value=memoize.get(
        KEY_POINTS_OF_INTEREST_IMPL_DROPDOWN,
        default=points_of_interest_impls_module_names[0],
        possible_values=points_of_interest_impls_module_names,
    ),
    description="Point of interest implementation",
    **widgets_styling,
)
reload_impl_button = wid.Button(
    description="Reload Implementation",
    **widgets_styling,
)
output = wid.Output()


@output.capture(clear_output=True, wait=True)
def on_menu_change(change=None):
    memoize.set(KEY_IMAGE_SET_DROPDOWN, image_set_dropdown.value)
    memoize.set(
        KEY_POINTS_OF_INTEREST_IMPL_DROPDOWN, points_of_interest_impl_dropdown.value
    )

    # reload the impl module
    current_points_of_interest_impl = points_of_interest_impl_dropdown.value
    points_of_interest_impl = dyn.load_module(current_points_of_interest_impl)

    input_image_set = input_image_sets[image_set_dropdown.value]
    total_image_count = len(input_image_set)
    KEY_IMAGE_COUNT_SLIDER = f"image_count_slider_{image_set_dropdown.value}"
    current_image_count_slider_value = memoize.get(
        KEY_IMAGE_COUNT_SLIDER, default=total_image_count
    )
    if current_image_count_slider_value > total_image_count:
        current_image_count_slider_value = total_image_count
    image_count_slider = wid.IntSlider(
        value=current_image_count_slider_value,
        min=1,
        max=total_image_count,
        step=1,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format="d",
        description="Image count",
    )

    def on_image_count_slider_change(change=None):
        memoize.set(KEY_IMAGE_COUNT_SLIDER, image_count_slider.value)
        on_menu_change()

    image_count_slider.observe(on_image_count_slider_change, names="value")
    display(image_count_slider)

    image_set = input_image_set[: image_count_slider.value]

    with LogTimer("Displaying input images"):
        input_ipy_images_color = [
            to_ipy_image(image.image_color, longest_side=image_size, upscale=True)
            for image in image_set
        ]
        display(wid.HBox(input_ipy_images_color))
        input_ipy_images_gray = [
            to_ipy_image(image.image_gray, longest_side=image_size, upscale=True)
            for image in image_set
        ]
        display(wid.HBox(input_ipy_images_gray))

    KEY_SIGMA1_SLIDER = "sigma1_slider"
    sigma1_slider = wid.FloatSlider(
        value=memoize.get(KEY_SIGMA1_SLIDER, default=0.8),
        min=0.1,
        max=20.0,
        step=0.1,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".1f",
        description="Sigma 1",
    )
    KEY_SIGMA2_SLIDER = "sigma2_slider"
    sigma2_slider = wid.FloatSlider(
        value=memoize.get(KEY_SIGMA2_SLIDER, default=1.5),
        min=0.1,
        max=20.0,
        step=0.1,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".1f",
        description="Sigma 2",
    )
    KEY_THRESHOLD_SLIDER = "threshold_slider"
    threshold_slider = wid.FloatSlider(
        value=memoize.get(KEY_THRESHOLD_SLIDER, default=0.01),
        min=0.0,
        max=0.1,
        step=0.01,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".2f",
        description="Threshold",
    )
    KEY_HARRIS_K_SLIDER = "harris_k_slider"
    harris_k_slider = wid.FloatSlider(
        value=memoize.get(KEY_HARRIS_K_SLIDER, default=0.04),
        min=0.01,
        max=0.1,
        step=0.01,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".2f",
        description="Harris k",
    )
    default_harris_button = wid.Button(
        description="Default Harris",
        **widgets_styling,
    )
    output_harris_corner = wid.Output()

    @output_harris_corner.capture(clear_output=True, wait=True)
    def on_harris_change(config=None):
        memoize.set(KEY_SIGMA1_SLIDER, sigma1_slider.value)
        memoize.set(KEY_SIGMA2_SLIDER, sigma2_slider.value)
        memoize.set(KEY_THRESHOLD_SLIDER, threshold_slider.value)
        memoize.set(KEY_HARRIS_K_SLIDER, harris_k_slider.value)

        image_gray_array = np.array([image.image_gray for image in image_set])
        image_color_array = np.array([image.image_color for image in image_set])

        with LogTimer("Calculating Harris corners"):
            harris_corner_keystones = points_of_interest_impl.harris_corner(
                image_gray_array,
                sigma1_slider.value,
                sigma2_slider.value,
                harris_k_slider.value,
                threshold_slider.value,
            )

        with LogTimer("Displaying Harris corners"):
            annotated_harris_ipy_images = []
            for image_idx, image in enumerate(image_set):
                keypoint_size = int(max(image.image_color.shape[:2]) / 512)
                for keypoint in harris_corner_keystones[image_idx]:
                    keypoint.size *= keypoint_size

                annotated_image = draw_keypoints(
                    image_color_array[image_idx],
                    harris_corner_keystones[image_idx],
                )
                annotated_harris_ipy_images.append(
                    wid.VBox(
                        [
                            wid.Label(f"{image.filename}"),
                            wid.Label(
                                f"Found {len(harris_corner_keystones[image_idx])} Harris corners"
                            ),
                            to_ipy_image(
                                annotated_image, longest_side=image_size, upscale=True
                            ),
                        ]
                    )
                )
            display(wid.HTML("<h2>Harris corners</h2>"))
            display(wid.HBox(annotated_harris_ipy_images))

            if total_image_count < 2:
                display(wid.HTML("<h2>Not enough images to calculate matches</h2>"))
                return
            # Flann based matcher
            display(wid.HTML("<h2>Flann matches</h2>"))
            KEY_FLANN_IMAGE_1_DROPDOWN = "flann_image_1_dropdown"
            flann_image_1_dropdown = wid.Dropdown(
                options=[image.filename for image in image_set],
                value=memoize.get(
                    KEY_FLANN_IMAGE_1_DROPDOWN,
                    default=image_set[0].filename,
                    possible_values=[image.filename for image in image_set],
                ),
                description="Image 1",
                **widgets_styling,
            )
            KEY_FLANN_IMAGE_2_DROPDOWN = "flann_image_2_dropdown"
            flann_image_2_dropdown = wid.Dropdown(
                options=[image.filename for image in image_set],
                value=memoize.get(
                    KEY_FLANN_IMAGE_2_DROPDOWN,
                    default=image_set[1].filename,
                    possible_values=[image.filename for image in image_set],
                ),
                description="Image 2",
                **widgets_styling,
            )
            KEY_PATCH_SIZE_SLIDER = "patch_size_slider"
            patch_size_slider = wid.IntSlider(
                value=memoize.get(KEY_PATCH_SIZE_SLIDER, default=5),
                min=1,
                max=15,
                step=1,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format="d",
                description="Patch size",
            )
            default_flann_button = wid.Button(
                description="Default flann values",
                **widgets_styling,
            )
            output_flann = wid.Output()

            @output_flann.capture(clear_output=True, wait=True)
            def on_flann_change(config=None):
                memoize.set(KEY_FLANN_IMAGE_1_DROPDOWN, flann_image_1_dropdown.value)
                memoize.set(KEY_FLANN_IMAGE_2_DROPDOWN, flann_image_2_dropdown.value)
                memoize.set(KEY_PATCH_SIZE_SLIDER, patch_size_slider.value)

                image_1_idx = [image.filename for image in image_set].index(
                    flann_image_1_dropdown.value
                )
                image_2_idx = [image.filename for image in image_set].index(
                    flann_image_2_dropdown.value
                )

                image_gray_1 = image_gray_array[image_1_idx]
                image_gray_2 = image_gray_array[image_2_idx]

                keypoints_1 = harris_corner_keystones[image_1_idx]
                keypoints_2 = harris_corner_keystones[image_2_idx]

                with LogTimer("Compute Descriptors"):
                    filtered_keypoints_1, descriptors_1 = (
                        points_of_interest_impl.compute_descriptors(
                            image_gray_1, keypoints_1, patch_size_slider.value
                        )
                    )
                    filtered_keypoints_2, descriptors_2 = (
                        points_of_interest_impl.compute_descriptors(
                            image_gray_2, keypoints_2, patch_size_slider.value
                        )
                    )

                with LogTimer("Calculating Flann matches"):
                    matches = points_of_interest_impl.flann_matches(
                        descriptors_1, descriptors_2
                    )

                with LogTimer("Filtering Flann matches"):
                    matches_filtered = points_of_interest_impl.filter_matches(matches)

                with LogTimer("Drawing Flann matches"):
                    draw_matches_image = draw_matches(
                        image_color_array[image_1_idx],
                        filtered_keypoints_1,
                        image_color_array[image_2_idx],
                        filtered_keypoints_2,
                        matches_filtered,
                    )

                with LogTimer("Displaying Flann matches"):
                    display(
                        to_ipy_image(
                            draw_matches_image,
                            longest_side=image_size,
                            upscale=True,
                        )
                    )

            flann_image_1_dropdown.observe(on_flann_change, names="value")
            flann_image_2_dropdown.observe(on_flann_change, names="value")
            patch_size_slider.observe(on_flann_change, names="value")

            def default_flann(change=None):
                memoize.delete_keys(
                    [
                        KEY_FLANN_IMAGE_1_DROPDOWN,
                        KEY_FLANN_IMAGE_2_DROPDOWN,
                        KEY_PATCH_SIZE_SLIDER,
                    ]
                )
                on_menu_change()

            default_flann_button.on_click(default_flann)

            display(
                wid.VBox(
                    [
                        wid.HBox([flann_image_1_dropdown, flann_image_2_dropdown]),
                        wid.HBox([patch_size_slider, default_flann_button]),
                        output_flann,
                    ]
                )
            )
            on_flann_change()

    sigma1_slider.observe(on_harris_change, names="value")
    sigma2_slider.observe(on_harris_change, names="value")
    threshold_slider.observe(on_harris_change, names="value")
    harris_k_slider.observe(on_harris_change, names="value")

    def default_harris(change=None):
        memoize.delete_keys(
            [
                KEY_SIGMA1_SLIDER,
                KEY_SIGMA2_SLIDER,
                KEY_THRESHOLD_SLIDER,
                KEY_HARRIS_K_SLIDER,
            ]
        )
        on_menu_change()

    default_harris_button.on_click(default_harris)

    display(
        wid.VBox(
            [
                sigma1_slider,
                sigma2_slider,
                threshold_slider,
                harris_k_slider,
                default_harris_button,
                output_harris_corner,
            ]
        )
    )
    on_harris_change()


image_set_dropdown.observe(on_menu_change, names="value")
points_of_interest_impl_dropdown.observe(on_menu_change, names="value")
reload_impl_button.on_click(on_menu_change)

display(
    wid.VBox(
        [
            wid.HBox(
                [
                    image_set_dropdown,
                    points_of_interest_impl_dropdown,
                    reload_impl_button,
                ]
            ),
            output,
        ]
    )
)
on_menu_change()